In [1]:
import pandas as pd
import numpy as np 
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense 
from keras import applications 
from keras.utils.np_utils import to_categorical 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math 
import datetime
import time
import queue
from multiprocessing import Queue

Using TensorFlow backend.


In [2]:
#Default dimensions we found online
img_width, img_height = 224, 224 
 
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model.h5'
# loading up our datasets
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'
 
# number of epochs to train top model 
epochs = 7 #this has been changed after multiple model run 
# batch size used by flow_from_directory and predict_generator 
batch_size = 50 

In [3]:
#Loading vgc16 model
vgg16 = applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255) 
#needed to create the bottleneck .npy files

W0912 15:12:11.385747 12100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0912 15:12:11.397787 12100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0912 15:12:11.401747 12100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0912 15:12:11.425750 12100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0912 15:12:11.693299 12100 deprecation_wrapper.py:119] From C:\ProgramData\Ana

In [4]:
'''# Train

#__this can take an hour and half to run so only run it once. 
#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__
start = datetime.datetime.now()
 
generator = datagen.flow_from_directory( 
    train_data_dir, 
    target_size=(img_width, img_height), 
    batch_size=batch_size, 
    class_mode=None, 
    shuffle=False) 
 
nb_train_samples = len(generator.filenames) 
num_classes = len(generator.class_indices) 
 
predict_size_train = int(math.ceil(nb_train_samples / batch_size)) 
 
bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) 
 
np.save('bottleneck_features_train.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)'''

"# Train\n\n#__this can take an hour and half to run so only run it once. \n#once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__\nstart = datetime.datetime.now()\n \ngenerator = datagen.flow_from_directory( \n    train_data_dir, \n    target_size=(img_width, img_height), \n    batch_size=batch_size, \n    class_mode=None, \n    shuffle=False) \n \nnb_train_samples = len(generator.filenames) \nnum_classes = len(generator.class_indices) \n \npredict_size_train = int(math.ceil(nb_train_samples / batch_size)) \n \nbottleneck_features_train = vgg16.predict_generator(generator, predict_size_train) \n \nnp.save('bottleneck_features_train.npy', bottleneck_features_train)\nend= datetime.datetime.now()\nelapsed= end-start\nprint ('Time: ', elapsed)"

In [5]:
#Validation

#__this can take half an hour to run so only run it once. once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__\n",
    
start = datetime.datetime.now()
generator = datagen.flow_from_directory(  
         validation_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
       
nb_validation_samples = len(generator.filenames)  
       
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size)) 
       
bottleneck_features_validation = vgg16.predict_generator(  
         generator, predict_size_validation)  
       
np.save('bottleneck_features_validation.npy', bottleneck_features_validation)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Found 4352 images belonging to 51 classes.
Time:  0:18:26.282737


In [6]:
#Test

#__this can take half an hour to run so only run it once. once the npy files have been created, no need to run again. Convert this cell to a code cell to run.__\n",
    
start = datetime.datetime.now()
generator = datagen.flow_from_directory(  
         test_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
       
nb_test_samples = len(generator.filenames)  
       
predict_size_test = int(math.ceil(nb_test_samples / batch_size))  
       
bottleneck_features_test = vgg16.predict_generator(  
         generator, predict_size_test)  
       
np.save('bottleneck_features_test.npy', bottleneck_features_test) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Found 4945 images belonging to 51 classes.
Time:  0:20:26.303864


In [7]:


#training data
generator_top = datagen.flow_from_directory( 
   train_data_dir, 
   target_size=(img_width, img_height), 
   batch_size=batch_size, 
   class_mode='categorical', 
   shuffle=False) 
 
nb_train_samples = len(generator_top.filenames) 
num_classes = len(generator_top.class_indices) 
 
# load the bottleneck features saved earlier 
train_data = np.load('bottleneck_features_train.npy') 
 
# get the class labels for the training data, in the original order 
train_labels = generator_top.classes 
 
# convert the training labels to categorical vectors 
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 20123 images belonging to 51 classes.


In [8]:


#validation data\n",
generator_top = datagen.flow_from_directory(
             validation_data_dir,  
             target_size=(img_width, img_height),  
             batch_size=batch_size,  
             class_mode=None,  
             shuffle=False)  
       
nb_validation_samples = len(generator_top.filenames)  
       
validation_data = np.load('bottleneck_features_validation.npy')  
       
    
validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

Found 4352 images belonging to 51 classes.


In [9]:


#testing data\n",
generator_top = datagen.flow_from_directory(  
             test_data_dir,  
             target_size=(img_width, img_height),  
             batch_size=batch_size,  
             class_mode=None,  
             shuffle=False)  
       
nb_test_samples = len(generator_top.filenames)  
       
test_data = np.load('bottleneck_features_test.npy')  
       
    
test_labels = generator_top.classes 
test_labels = to_categorical(test_labels, num_classes=num_classes)

Found 4945 images belonging to 51 classes.


In [10]:
#This is the best model we found. For additional models, check out I_notebook.ipynb
start = datetime.datetime.now()
model = Sequential() 
model.add(Flatten(input_shape=train_data.shape[1:])) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.5)) 
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3))) 
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
   optimizer=optimizers.RMSprop(lr=1e-4),
   metrics=['acc'])
history = model.fit(train_data, train_labels, 
   epochs=7,
   batch_size=batch_size, 
   validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path)
(eval_loss, eval_accuracy) = model.evaluate( 
    validation_data, validation_labels, batch_size=batch_size,     verbose=1)
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100)) 
#print("[INFO] Loss: {}".format(eval_loss)) 
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

C:\ProgramData\Anaconda3\lib\site-packages\keras\activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
W0912 15:51:45.927205 12100 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0912 15:51:46.383285 12100 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



ValueError: Input arrays should have the same number of samples as target arrays. Found 20136 input samples and 20123 target samples.

In [ ]:
#Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy') 
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss') 
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
# Summary of neural network

model.summary()

In [ ]:
take = queue.Queue()

In [ ]:
def read_image(file_path):
   print("[INFO] loading and preprocessing image…") 
   image = load_img(file_path, target_size=(224, 224)) 
   image = img_to_array(image) 
   image = np.expand_dims(image, axis=0)
   image /= 255. 
   return image

def test_single_image(path):
  animals = ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 
            'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 
            'শ', 'ষ', 'স', 'হ', 'ড়', 'ঢ়', 'য়', 'া', 'ে', 'ু', 'ূ', 'ৃ', '০', '১', '২', '৩', '৪', '৫', '৬', '৭',
             '৮','৯']
  images = read_image(path)
  time.sleep(.5)


  bt_prediction = vgg16.predict(images) 
  preds = model.predict_proba(bt_prediction)
  per = 0
  for idx, animal, x in zip(range(0, 61), animals , preds[0]):
   if (round (x*100, 2) > per):
    per = round (x*100, 2)
    container = animal
    print ('Here = ', container, per)
    
   print("ID: {}, Label: {} {}%".format(idx, animal, round(x*100,2) ))


  print('Final Decision:')
  time.sleep(.5)
  for x in range(3):
   print('.'*(x+1))
   time.sleep(.2)
  class_predicted = model.predict_classes(bt_prediction)
  class_dictionary = generator_top.class_indices 
  inv_map = {v: k for k, v in class_dictionary.items()}

  if (container == 'ু'):
    container = 'ড়'
  elif (container == 'ূ'):
    container = 'ঢ়'
  elif (container == 'ৃ'):
    container = 'য়'
  elif (container == 'ড়'):
    container = 'া'
  elif (container == 'ঢ়'):
    container = 'ু'
  elif (container == 'য়'):
    container = 'ূ'
  elif (container == 'া'):
    container = 'ৃ' 

  print("Character: {}, Label: {}, Accuracy: {}%".format(class_predicted[0],  container, per))
  #if (per >= 30):
  print("ID: {}, Label: {}, Accuracy: {}%".format(class_predicted[0],  container, per))  
  take.put (container)
  return load_img(path)
path = (r'C:\Users\th4pa\Bangla\test - Copy (4).png')
test_single_image(path)

In [ ]:
word = ""
while not take.empty():
    word = word + take.get()
    
print (word)